# check the files for the images

In [ ]:
import json
import sys
import hashlib
import glob
from tqdm import tqdm
from collections import defaultdict
import os
import magic

with open("hashes_1.json", "r") as f:
    hashes = json.loads(f.read())
hashes = set(hashes)
print(len(hashes))

with open("hashes_magic_dict_1.json", "r") as f:
    magic_dict = json.loads(f.read())

new_d = defaultdict(set)
for k, v in magic_dict.items():
    new_d[k] = set(v)
    
print(len(new_d))

links = glob.glob("../../../images/new/*")
alll = [(l, len(glob.glob(l + "/*"))) for l in links]

links = glob.glob("../../../images/new/00/*")
wrong = [i for i in links if "jpg" not in i]
wr = [(magic.from_file(w, mime=True), w) for w in wrong]
test = [w[0] for w in wr]
set(test)

# Some statistics code 

In [ ]:
from node import Node
from babelnetgraph import BabelNetGraph
from relation import Relation
import urllib.request
import json
from utils import from_lemma_to_ids, create_folder, from_synsetID_to_images, get_edges_from_synset, return_core_graph, process_sense_info, edge_information, process_imgs, get_key
from tqdm import tqdm
import time 
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
from collections import Counter, defaultdict
import os
import subprocess

# OR 180k!!
with open("../data/new_mappings/try1/nodes_100k.json", "r") as f:
    nodes = json.loads(f.read())
    
with open("../data/new_mappings/try1/edges_100k_small.json", "r") as f:
    all_edges = json.loads(f.read())

def return_statistics(nodes, all_edges):
    node_lengths = len(nodes.keys())
    sum_edges = 0
    set_rels = set()
    raw_rels = []
    avg_edg = 0
    for i in all_edges.values():
        sum_edges += len(i)
        rels = [el["r"] for el in i]
        set_rels.update(rels)
        raw_rels.extend(rels)
        avg_edg += len(rels)
    distr_rels = dict(Counter(raw_rels))
    avg_distr_rels = {k: v/node_lengths for k, v in distr_rels.items()}
    raw_rels = None
    avg_edg /= node_lengths
    
    avg_sense = 0
    avg_gloss = 0
    for i in nodes.keys():
        val = nodes[i]
        avg_sense += len(val["se"])
        avg_gloss += len(val["gl"])
    avg_sense /= node_lengths
    avg_gloss /= node_lengths
    
    print("Number of Nodes: ")
    print(len(nodes.keys()))
    
    print("Number of Edges: ")
    print(sum_edges)
    
    print("Number of Unique Relations: ")
    print(len(set_rels))
    print(set_rels)
    
    print("Distribution Relations: ")
    print(distr_rels)
    
    print("Avg Amount of Senses per Node: ")
    print(avg_sense)
    
    print("Avg Amount of Glosses per Node: ")
    print(avg_gloss)
    
    print("Avg Amount of Edges per Node: ")
    print(avg_edg)
    
    print("Avg Type of Edges per Node: ")
    print(avg_distr_rels)
    
    G = nx.DiGraph()
    G.add_edges_from([(i, edg["s"]) for i, j in all_edges.items() for edg in j])
    print("Connectivity")
    print([len(i) for i in nx.weakly_connected_components(G)])
    
    plot_distr(distr_rels)
    
def plot_distr(distr_rels):
    x = []
    y = []
    for k, v in distr_rels.items():
        x.append(k)
        y.append(v)

    x_pos = np.arange(len(x))

    # Build the plot
    fig, ax = plt.subplots()
    ax.bar(x_pos, y, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax.set_ylabel('Amount of relation')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(x)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    #ax.set_title('Coefficent of Thermal Expansion (CTE) of Three Metals')
    ax.yaxis.grid(True)

    # Save the figure and show
    plt.tight_layout()
    plt.savefig('../data/imgs_out/distr_180k_small.png')
    plt.show()